In [32]:
import pandas as pd
import numpy as np
from whoswho import who
from rapidfuzz import process

In [33]:
def stringToList(x):
    return x.strip('][\"\'').split(', ') if not pd.isnull(x) else x

In [34]:
matching = pd.read_csv("Data/final_matching.csv", index_col = 0)
matching_post1790 = pd.read_csv("Data/final_matching_post1790.csv", index_col = 0)

In [35]:
all_matches_post1790 = pd.read_csv('Data/total_matching_post1790.csv', index_col = 0)

In [36]:
matching_merged = pd.concat([matching, matching_post1790]).drop_duplicates()
matching_merged = matching_merged[matching_merged['Scores'] != 100][['Delegates', 'Loan Matches']]
matching_merged.columns = ['Original Name', 'Matches']
all_matches_post1790.columns = ['Original Name', 'Matches']
final_matching = pd.concat([matching_merged, all_matches_post1790], axis = 0)
final_matching.reset_index(drop = True, inplace = True)

In [37]:
final_matching.to_csv('ML_Proj/matching.csv')